# Data Science Challenge

Este código extrae info de los productos de la plataforma de Mercado Libre México para el analisis exploratorio de los productos con descuentos

In [1]:
import pandas as pd
import requests
import warnings
import os
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

Vemos las categorías que se tienen para la plataforma en México

In [2]:
import requests

# Obtener la lista de categorías para Mercado Libre México
def get_mexico_categories():
    url = "https://api.mercadolibre.com/sites/MLM/categories"
    response = requests.get(url)
    if response.status_code == 200:
        categories = response.json()
        return [(category['id'], category['name']) for category in categories]
    else:
        print("Error al obtener las categorías:", response.status_code)
        return []

# Ejemplo de uso
categories = get_mexico_categories()
print(f"Total de categorías: {len(categories)}")


Total de categorías: 31


Está función a continuación nos permite extraer los datos de la API para el caso de México que nos permitirá hacer el analisis exploratorio de descuentos

In [ ]:
# Función para obtener productos por rango de precios y condición
def fetch_products_by_price_and_condition(category_id, min_price, max_price, condition, max_pages=20):
    base_url = "https://api.mercadolibre.com/sites/MLM/search"
    all_results = []
    items_per_page = 50
    
    for page in range(max_pages):
        params = {
            'category': category_id,
            'price': f"{min_price}-{max_price}",
            'condition': condition,
            'limit': items_per_page,
            'offset': page * items_per_page,
        }
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            results = data.get('results', [])
            for product in results:
                # Extraer los detalles relevantes incluyendo sold_quantity
                product_info = {
                    'id': product.get('id'),
                    'title': product.get('title'),
                    'price': product.get('price'),
                    'sold_quantity': product.get('sold_quantity'),
                    'available_quantity': product.get('available_quantity'),
                    'condition': product.get('condition'),
                    'permalink': product.get('permalink'),
                    'thumbnail': product.get('thumbnail'),
                    'currency_id': product.get('currency_id'),
                    'category_id': product.get('category_id'),
                    'listing_type_id': product.get('listing_type_id'),
                    'start_time': product.get('start_time'),
                    'stop_time': product.get('stop_time'),
                    'shipping': product.get('shipping'),
                    'seller_id': product['seller'].get('id') if product.get('seller') else None,
                    'seller_power_seller_status': product['seller'].get('power_seller_status') if product.get('seller') else None,
                    'seller_registration_date': product['seller'].get('registration_date') if product.get('seller') else None,
                    'video_id': product.get('video_id'),
                    'warranty_time': product.get('warranty_time'),
                    'health': product.get('health'),
                    'pictures': product.get('pictures'),
                    'descriptions': product.get('descriptions'),
                    'sub_status': product.get('sub_status'),
                    'site_id': product.get('site_id'),
                    'geolocation': product.get('geolocation'),
                    'international_delivery_mode': product.get('international_delivery_mode'),
                    'catalog_product_id': product.get('catalog_product_id'),
                    'secure_url': product.get('secure_url'),
                    'automatic_relist': product.get('automatic_relist'),
                    'accepts_mercadopago': product.get('accepts_mercadopago'),
                    'deals': product.get('deals'),
                    'deal_ids': product.get('deal_ids'),
                    'seller_address': product.get('seller_address'),
                    'tags': product.get('tags'),
                    'subtitle': product.get('subtitle'),
                    'logistic_type': product.get('logistic_type'),
                    'warranty_type': product.get('warranty_type'),
                    'seller_custom_field': product.get('seller_custom_field'),
                    'variations': product.get('variations'),
                    'sold_quantity_by_period': product.get('sold_quantity_by_period'),
                    'order_backend': product.get('order_backend'),
                    'thumbnail_url': product.get('thumbnail_url'),
                    'differential_pricing_id': product.get('differential_pricing_id'),
                    'stock': product.get('stock'),
                    'installments.rate': product['installments'].get('rate') if product.get('installments') else None,
                    'installments.amount': product['installments'].get('amount') if product.get('installments') else None,
                    'installments.quantity': product['installments'].get('quantity') if product.get('installments') else None,
                }
    
                all_results.append(product_info)
            
            if len(results) < items_per_page:
                break
        else:
            print(f"Error al obtener datos para categoría {category_id}: {response.status_code}")
            break
            
    return all_results

# Función para iterar sobre categorías y almacenar datos por categoría
def fetch_and_store_all_categories_products(categories, price_ranges, conditions, max_pages=20):
    for category_id, category_name in categories:
        print(f"Fetching data for category: {category_name} (ID: {category_id})")
        
        category_data = []
        
        for min_price, max_price in price_ranges:
            for condition in conditions:
                print(f" - Price range: {min_price}-{max_price}, Condition: {condition}")
                products = fetch_products_by_price_and_condition(category_id, min_price, max_price, condition, max_pages=max_pages)
                category_data.extend(products)
        
        # Convertir los datos a DataFrame
        df = pd.DataFrame(category_data)
        
        # Crear un directorio para almacenar los archivos por categoría
        category_dir = f"data/{category_name}"
        os.makedirs(category_dir, exist_ok=True)
        
        # Guardar los datos en un archivo CSV por categoría
        file_path = os.path.join(category_dir, f"{category_name}_products.csv")
        df.to_csv(file_path, index=False)
        print(f"Data for category {category_name} saved to {file_path}")

# Definir los rangos de precios y condiciones
price_ranges = [
    (0, 5000),
    (5001, 10000),
    (10001, 20000),
    (20001, 50000),
    (50001, 10000000),
]

conditions = ['new', 'used']

# Obtener la lista de categorías para Mercado Libre México
categories = get_mexico_categories()

# Obtener y almacenar productos por categorías con los criterios definidos
fetch_and_store_all_categories_products(categories, price_ranges, conditions)


Cargamos todos los datos en un solo CSV

In [ ]:


# Función para verificar si un archivo está vacío
def is_file_empty(file_path):
    return os.path.getsize(file_path) == 0

# Función para leer archivos CSV de cada categoría y combinarlos en un solo DataFrame
def load_and_combine_data(data_dir):
    combined_df = pd.DataFrame()  # DataFrame vacío para combinar todos los datos
    
    # Iterar sobre todas las carpetas en el directorio de datos
    for category_name in os.listdir(data_dir):
        category_dir = os.path.join(data_dir, category_name)
        
        if os.path.isdir(category_dir):  # Verifica si es un directorio
            # Buscar archivos CSV en la carpeta de la categoría
            for file_name in os.listdir(category_dir):
                if file_name.endswith('.csv'):
                    file_path = os.path.join(category_dir, file_name)
                    
                    # Verificar si el archivo está vacío
                    if is_file_empty(file_path):
                        print(f"Archivo vacío: {file_path}, saltando...")
                        continue
                    
                    try:
                        # Leer el archivo CSV
                        df = pd.read_csv(file_path)
                        
                        # Agregar una columna con el nombre de la categoría
                        df['category'] = category_name
                        
                        # Combinar con el DataFrame principal
                        combined_df = pd.concat([combined_df, df], ignore_index=True)
                    except pd.errors.EmptyDataError:
                        print(f"Error: El archivo {file_path} está vacío o no contiene datos válidos.")
    
    return combined_df

# Directorio donde están almacenados los archivos por categorías
data_dir = 'data'

# Cargar y combinar todos los datos
all_products_df = load_and_combine_data(data_dir)

# Opcional: Guardar el DataFrame combinado en un nuevo archivo CSV
all_products_df.to_csv('mercado_libre_all_combined.csv', index=False)


In [6]:
all_products_df.shape

(86287, 39)

In [7]:
all_products_df[lambda x:x.original_price.notnull()].shape[0]

5047

Leemos los datos que obtuvimos del otro repo en la carpeta data_prov y lo combinamos en uno solo

In [4]:
# Directorio
carpeta = os.path.expanduser('~/Documents/busqueda_chambi/2024/meli_no_repo/data_prov')
archivos_csv = [f for f in os.listdir(carpeta) if f.endswith('.csv')]

dataframes = []

# Progreso total
total_archivos = len(archivos_csv)

for i, archivo in enumerate(archivos_csv, start=1):
    try:
        df = pd.read_csv(os.path.join(carpeta, archivo), sep=';', encoding='utf-8')
        dataframes.append(df)
        print(f"[{i}/{total_archivos}] Archivo '{archivo}' cargado satisfactoriamente.")
    except Exception as e:
        print(f"[{i}/{total_archivos}] Error al cargar '{archivo}': {e}")

df_combinado = pd.concat(dataframes, ignore_index=True)
print("Todos los archivos se han combinado exitosamente.")


[1/31] Archivo 'MCO1368.csv' cargado satisfactoriamente.
[2/31] Archivo 'MCO3025.csv' cargado satisfactoriamente.
[3/31] Archivo 'MCO1168.csv' cargado satisfactoriamente.
[4/31] Archivo 'MCO1430.csv' cargado satisfactoriamente.
[5/31] Archivo 'MCO1747.csv' cargado satisfactoriamente.
[6/31] Archivo 'MCO1182.csv' cargado satisfactoriamente.
[7/31] Archivo 'MCO1384.csv' cargado satisfactoriamente.
[8/31] Archivo 'MCO1743.csv' cargado satisfactoriamente.
[9/31] Archivo 'MCO1540.csv' cargado satisfactoriamente.
[10/31] Archivo 'MCO5726.csv' cargado satisfactoriamente.
[11/31] Archivo 'MCO1144.csv' cargado satisfactoriamente.
[12/31] Archivo 'MCO118204.csv' cargado satisfactoriamente.
[13/31] Archivo 'MCO441917.csv' cargado satisfactoriamente.
[14/31] Archivo 'MCO1132.csv' cargado satisfactoriamente.
[15/31] Archivo 'MCO1051.csv' cargado satisfactoriamente.
[16/31] Archivo 'MCO1246.csv' cargado satisfactoriamente.
[17/31] Archivo 'MCO1459.csv' cargado satisfactoriamente.
[18/31] Archivo 'MC

In [ ]:
df_combinado.to_csv("df_model_comb.csv", index = False)

In [ ]:
import session_info
session_info.show(html=False)